In [6]:
#!pip install pandas_datareader
#!pip install yfinance

In [ ]:
https://www.kaggle.com/code/trooperog/ai-trading-bot

In [1]:
import math
import random
import numpy as np
import pandas as pd
from datetime import datetime


import tensorflow as tf
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()


from tqdm import tqdm_notebook, tqdm
from collections import deque

In [39]:
.995**170

0.4265046070983015

In [2]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    # Define hyperparamaters
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
        
    # Call a function  to build a model trought this class constructor
    # More parameters could be ustilized to programaticaly define network size (layers and neurons)
    self.model = self.model_builder()
    
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))    
    return model
  
  
  
  
  # Trade function that takes state as an input and returns an action 
  # to perform in perticular state 
  def trade(self, state):
    
    # Should we perform a renadom generated action or action defined in model?
    
    # If value from our random generator is smaller or equal to our epsilon 
    #     then we will retun a random action from action_space [0-3)
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    # If our random is greater than epsilon then we will use model to perform action
    actions = self.model.predict(state)
    # return only a one number defining an action (#Stay - 0 , Buy - 1, Sell - 2) 
    #    that has maximum probability
    return np.argmax(actions[0])
  
  
  
  def batch_train(self, batch_size):
    
    batch = []
    
    # Iterrate in momory, we do not want to randolmy select data as we are dealing with 
    #    time constraint data. We will always sample from the end of memory size of bath
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      # insert data from memory to batch      
      batch.append(self.memory[i])
    
    
    # Iterate trought batch of data and train the model for each sample from batch
    # Order of variables in for loop is important
    for state, action, reward, next_state, done in batch:
      # Reward if agent is in terminal state
      reward = reward
      # Check that agent is not in terminal state
      # If not in terminal state calculate reward for actions that could be played
      if not done:
        # Discounted total reward:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])        
      # Target variable that is predicted by the model (action)
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    # We will decrease epsilon parameter that is 1 as defined in __init__  so
    #    so we can stop performing random actions at some point
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [3]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

  
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))


def dataset_loader(stock_name):
  
  #Use pandas data reader for reading stock data from warious sources like "yahoo", "google"
  dataset = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2024-01-01")
   
  # Get start and end time to variables from dataset
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  # Model will use "Close" column for training 
  close = dataset['Close']

  return close

# Data -> dataset to predict from, gathered by data:loader()
# Timestep -> Day in the dataset that we want to predict for [0:datalength]
# window_suze -> how many days in past we want to use to predict current status[1:datalength]
#         Try different setup to see what creates best fit
def state_creator(data, timestep, window_size):
  
  #starting day of our state
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data.iloc[starting_id:timestep+1]
  else:
    #Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data.iloc[0]] + list(data[0:timestep+1])
    
  state = []
  #Iterate trough whole windowed_data minus current state (-1)
  for i in range(window_size - 1):
    #Normalize the difference from current day and the next day
    #Because the prices can be very different and we want them on same scale
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])


In [4]:
# Tage data for Apple
stock_name = "AAPL"
data = dataset_loader(stock_name)

[*********************100%***********************]  1 of 1 completed


In [13]:
data.head(10)

Date
2017-01-03    225.240005
2017-01-04    226.580002
2017-01-05    226.399994
2017-01-06    227.210007
2017-01-09    226.460007
2017-01-10    226.460007
2017-01-11    227.100006
2017-01-12    226.529999
2017-01-13    227.050003
2017-01-17    226.250000
Name: Close, dtype: float64

In [7]:
timestep = 0
window_size = 10

starting_id = timestep - window_size + 1
  
if starting_id >= 0:
    windowed_data = data.iloc[starting_id:timestep+1]
else:
    #Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data.iloc[0]] + list(data[0:timestep+1])

In [9]:
trader = AI_Trader(window_size)
trader.model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x282eb4d60>>

In [10]:
episode = 1
state = state_creator(data, 0 , window_size + 1)
total_profit = 0

trader.inventory = []

In [14]:
state

array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]])

In [15]:
window_size = 10
episodes = 1000 # same as epoch

batch_size = 32
data_samples = len(data) - 1 # discard last value, that we will predict on

In [16]:
data_samples

1723

In [17]:
timestep = 15
window_size = 10
  
#starting day of our state
starting_id = timestep - window_size + 1
  
if starting_id >= 0:
   windowed_data = data.iloc[starting_id:timestep+1]
else:
    #Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data.iloc[0]] + list(data[0:timestep+1])
    
state = []
#Iterate trough whole windowed_data minus current state (-1)
for i in range(window_size - 1):
    #Normalize the difference from current day and the next day
    #Because the prices can be very different and we want them on same scale
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
np.array([state])

array([[0.36123513, 0.62714877, 0.31002487, 0.62245933, 0.30153556,
        0.69635532, 0.35663219, 0.81000031, 0.87761124]])

In [18]:
sigmoid(windowed_data[8+1] windowed_data[8+1])

0.8776112442924269

In [22]:
sigmoid(windowed_data[8+1]- windowed_data[8])

0.8776112442924269

In [18]:
windowed_data

Date
2017-01-11    227.100006
2017-01-12    226.529999
2017-01-13    227.050003
2017-01-17    226.250000
2017-01-18    226.750000
2017-01-19    225.910004
2017-01-20    226.740005
2017-01-23    226.149994
2017-01-24    227.600006
2017-01-25    229.570007
Name: Close, dtype: float64

In [19]:
data

Date
2017-01-03    225.240005
2017-01-04    226.580002
2017-01-05    226.399994
2017-01-06    227.210007
2017-01-09    226.460007
                 ...    
2023-10-24    423.630005
2023-10-25    417.549988
2023-10-26    412.549988
2023-10-27    410.679993
2023-10-30    415.589996
Name: Close, Length: 1718, dtype: float64

In [26]:
episode = 2
state = state_creator(data, 0 , window_size + 1)
state

array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]])

In [27]:
len(data)

1718

In [ ]:
episode = 1
state = state_creator(data, 0 , window_size + 1)
total_profit = 0

trader.inventory = []

for t in tqdm(range(len(data))):
    action = trader.trade(state)

        # Use action to get to next state(t+)
    next_state = state_creator(data, t+1, window_size + 1)
    # As we did not calculate anything up to this point reward is 0
    reward = 0
    
    if action == 1: #Buying
      # Put buyed stock to inventory to trade with
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    # To sell we need to have something in inventory  
    elif action == 2 and len(trader.inventory) > 0: #Selling
      # Check buy price, pop removes first value from list
      buy_price = trader.inventory.pop(0)
      
      # If we gain money (current price - buy price) we have reward 
      #    if we lost money then reward is 0
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    # if t is last sample in our dateset we are done
    #     we do not have any steps to perform in current episode
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    # Append all data to trader-agent memory, experience buffer
    trader.memory.append((state, action, reward, next_state, done))
    
    # change state to next state, so we are done with an episode
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    # Chekc if we have more information in our memory than batch size
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
  
  # Save the model every 10 episodes
    if episode % 10 == 0:
       trader.model.save("ai_trader_{}.h5".format(episode))


  0%|                                                                                                              | 0/1724 [00:00<?, ?it/s]

AI Trader bought:  $ 226.460007
AI Trader sold:  $ 227.100006  Profit: $ 0.639999
AI Trader bought:  $ 226.529999
AI Trader bought:  $ 227.050003
AI Trader sold:  $ 226.250000  Profit: - $ 0.279999
AI Trader sold:  $ 226.750000  Profit: - $ 0.300003
AI Trader bought:  $ 225.910004
AI Trader bought:  $ 226.149994
AI Trader bought:  $ 227.600006
AI Trader bought:  $ 229.330002
AI Trader bought:  $ 228.970001
AI Trader bought:  $ 227.550003
AI Trader bought:  $ 227.529999
AI Trader sold:  $ 227.770004  Profit: $ 1.860001
AI Trader bought:  $ 229.339996
AI Trader bought:  $ 228.929993
AI Trader bought:  $ 228.940002
AI Trader bought:  $ 229.240005
AI Trader sold:  $ 230.600006  Profit: $ 4.450012
AI Trader sold:  $ 231.509995  Profit: $ 3.909988
AI Trader bought:  $ 234.919998
AI Trader bought:  $ 235.089996
1/1 [==============================] - 0s 120ms/step


2023-11-07 17:05:59.209769: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 12ms/step


  2%|█▉                                                                                                   | 33/1724 [00:02<02:33, 11.03it/s]

...

1/1 [==============================] - 0s 9ms/step
AI Trader sold:  $ 294.350006  Profit: - $ 1.519989
1/1 [==============================] - 0s 9ms/step


 40%|████████████████████████████████████████▎                                                           | 694/1724 [24:43<33:44,  1.97s/it]

1/1 [==============================] - 0s 9ms/step


 40%|████████████████████████████████████████▎                                                           | 695/1724 [24:45<33:40,  1.96s/it]

1/1 [==============================] - 0s 9ms/step


In [28]:
trader.memory

deque([(array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]),
        1,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        False),
       (array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]),
        0,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        0,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.79248934, 0.45511914]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 

In [29]:
batch_size

32

In [32]:
trader.memory[100]

(array([[0.40131014, 0.78918293, 0.44522046, 0.01392572, 0.72111456,
         0.82346375, 0.77030014, 0.62948283, 0.63645198, 0.7595098 ]]),
 1,
 0,
 array([[0.78918293, 0.44522046, 0.01392572, 0.72111456, 0.82346375,
         0.77030014, 0.62948283, 0.63645198, 0.7595098 , 0.48750565]]),
 False)

In [74]:
data

Date
2017-01-03    225.240005
2017-01-04    226.580002
2017-01-05    226.399994
2017-01-06    227.210007
2017-01-09    226.460007
                 ...    
2023-10-20    421.190002
2023-10-23    420.459991
2023-10-24    423.630005
2023-10-25    417.549988
2023-10-26    412.549988
Name: Close, Length: 1716, dtype: float64

In [33]:
state

array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]])

In [71]:
trader.model.predict(state)

1/1 [==============================] - 0s 230ms/step


array([[45.306393, 43.670013, 46.8499  ]], dtype=float32)

In [89]:
state

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

In [34]:
trader.trade(state)

1/1 [==============================] - 0s 19ms/step


2

In [35]:
trader.memory

deque([(array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]),
        1,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        False),
       (array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]),
        0,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.79248934]]),
        0,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.79248934, 0.45511914]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 